ROHAN SARASWAT, PRN- 21070126071
AIML-B1. NLP ASSIGNMENT-1

#### Starting with importing Neccessary Libraries

In [ ]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/DATASETS/IMDB Dataset.csv')

In [ ]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
textdata=data['review']

In [ ]:
textdata

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [ ]:
import re
import string

# Cleaning of text data by performing neccessary procedures
def clean_text(text):
    # remove punctuations mark
    text = ''.join(word for word in text if word not in string.punctuation)
    text=''.join(word for word in text if word not in stopwords)
    text=re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    #text = text.translate(str.maketrans('', '', string.punctuation))
    # remove digits from data
    text01 = ''.join(w.lower() for w in text if not w.isdigit())
    # lower case of the data
    #text2 = text1.lower()
    # remove escape characters
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text01 = REPLACE_BY_SPACE_RE.sub(' ',text01)
    return text01

In [ ]:
# Lemmatization of text data
def lemmatize_text(text):
    wordlist = []
    lemmatizer = WordNetLemmatizer()
    # Sentence tokenization
    sentences = nltk.sent_tokenize(str(text))

    intial_sentences = sentences[0:1]
    final_sentences = sentences[len(sentences)-2: len(sentences)-1]

    # Word tokenization on data
    for sentence in intial_sentences:
        words = nltk.word_tokenize(sentence)
        for word in words:
            wordlist.append(word)

    for sentence in final_sentences:
        words = nltk.word_tokenize(sentence)
        for word in words:
            wordlist.append(word)

    return wordlist

In [ ]:
# Apply the functions on transcription column

# Clean
textdata = textdata.apply(lambda x: clean_text(x))
# Lemmatize
textdata = textdata.apply(lambda x: lemmatize_text(x))

In [ ]:
textdata

0        [one, f, he, her, revewer, h, enne, h, fer, wc...
1        [a, wnerful, lle, prucn, br, br, the, flng, ec...
2        [i, hugh, h, w, wnerful, w, pen, e, n, h, uer,...
3        [bcll, here, fl, where, lle, b, jke, hnk, here...
4        [peer, me, lve, n, he, te, f, mne, vull, unnng...
                               ...                        
49995    [i, hugh, h, ve, wn, rgh, g, jb, i, wn, creve,...
49996    [b, pl, b, lgue, b, cng, c, recng, he, nnng, p...
49997    [i, chlc, ugh, n, prchl, eleenr, chl, b, nun, ...
49998    [i, gng, hve, gree, wh, he, prevu, cen, n, e, ...
49999    [n, ne, expec, he, sr, trek, ve, be, hgh, r, b...
Name: review, Length: 50000, dtype: object

In [ ]:
# Performing Vectorization: ON the Data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# Text to TFIDF values conversion:
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english',
                             ngram_range=(1,3), max_df=0.75,
                             min_df=5, use_idf=True,
                             smooth_idf=True,max_features=5000)

# Fit and transform
tfIdfMat  = vectorizer.fit_transform(" ".join(x) for x in textdata)

# Feature names
feature_names = vectorizer.get_feature_names_out()
print(feature_names)

['abb' 'abc' 'abu' ... 'zne' 'zng' 'zngl']


In [ ]:
# Performance measure For measures report
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Including neccessary Algorithms:
algos=[('LR',LogisticRegression()),('SVM',SVC()),('RF',RandomForestClassifier())]

In [ ]:
# Defining an function for the data:
def nlpPipeline(algorithms,Xtrain,ytrain,Xtest,ytest):
  for algo,classifier in algorithms:
    pipeline=Pipeline([
        ("classifier",classifier)
    ])
    pipeline.fit(Xtrain,ytrain)
    ypredict=pipeline.predict(Xtest)
    print("Classification Report for "+algo)
    print(classification_report(y_test,
                                ypredict,

                                zero_division=0))

In [ ]:
# LabelEncoding y
y=data['sentiment'].replace('negative',0).replace('positive',1)

In [ ]:
y

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [ ]:
#train test spliit of thr tfidf dataset:
X_train, X_test, y_train, y_test = train_test_split(tfIdfMat,
                                                    y,
                                                    stratify=y,
                                                    random_state=1)

In [ ]:
# Printing the results:
print('Classification with TFIDF')
nlpPipeline(algos,X_train,y_train,X_test,y_test)

Classification with TFIDF
Classification Report for LR
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      6250
           1       0.86      0.86      0.86      6250

    accuracy                           0.86     12500
   macro avg       0.86      0.86      0.86     12500
weighted avg       0.86      0.86      0.86     12500

Classification Report for SVM
              precision    recall  f1-score   support

           0       0.87      0.86      0.87      6250
           1       0.86      0.87      0.87      6250

    accuracy                           0.87     12500
   macro avg       0.87      0.87      0.87     12500
weighted avg       0.87      0.87      0.87     12500

Classification Report for RF
              precision    recall  f1-score   support

           0       0.80      0.83      0.82      6250
           1       0.83      0.80      0.81      6250

    accuracy                           0.81     12500
   macro avg  

In [ ]:
# Importing another imp libraries
mport gensim
import numpy as np

In [ ]:
# Importing the google vector model
model=gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin',
                                                      binary=True)


In [ ]:
#vocabulary Collection:
vocab=model.key_to_index
print(len(vocab))

3000000


In [ ]:
# Creating an Embeddings function:
def embeddings(text):
  DIM=300
  zero_vec=np.zeros(DIM)
  features=[]
  for tokens in text:
    feat=np.zeros(DIM)
    count=0+1e-5 #for DIV by 0 error
    for token in tokens:
      if token in model:
        feat+=model[token]
        count +=1
    if(count !=0):
      features.append(feat/count)
    else:
      features.append(zero_vec)
  return features


In [ ]:
data_vec=embeddings(textdata)
print(len(data_vec))

50000


In [ ]:
# Spliting the data for Classifiction of word2Vec:
Xw2v_train,Xw2v_test,yw2v_train,yw2v_test=train_test_split(data_vec,y,stratify=y,random_state=1)

In [ ]:
print('Classification with the help of word2Vec')
nlpPipeline(algos,Xw2v_train,yw2v_train,Xw2v_test,yw2v_test)

Classification eith word2Vec


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report for LR
              precision    recall  f1-score   support

           0       0.71      0.70      0.71      6250
           1       0.71      0.72      0.71      6250

    accuracy                           0.71     12500
   macro avg       0.71      0.71      0.71     12500
weighted avg       0.71      0.71      0.71     12500

Classification Report for SVM
              precision    recall  f1-score   support

           0       0.72      0.72      0.72      6250
           1       0.72      0.72      0.72      6250

    accuracy                           0.72     12500
   macro avg       0.72      0.72      0.72     12500
weighted avg       0.72      0.72      0.72     12500

Classification Report for RF
              precision    recall  f1-score   support

           0       0.64      0.64      0.64      6250
           1       0.64      0.64      0.64      6250

    accuracy                           0.64     12500
   macro avg       0.64      0.64      0

### Using Count Vectorizer Model in Dataset

In [ ]:
# Assigining Values:
text=data['review']
text = text.apply(lambda x: clean_text(x))

In [ ]:
# assigining the values:
from sklearn.feature_extraction.text import CountVectorizer
vector=CountVectorizer()
mat=vector.fit_transform(text)

In [ ]:
vector.get_feature_names_out()

array(['aa', 'aaa', 'aaaaaaaaaaaahhhhhhhhhhhhhh', ..., 'þór', 'יגאל',
       'כרמון'], dtype=object)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(mat,y,stratify=y,random_state=1)

In [ ]:
# Printing the Report:

print('Classification with Count Vectorizer')
nlpPipeline(algos,X_train,y_train,X_test,y_test)

Classification with count vectorizer


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report for LR
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      6250
           1       0.86      0.86      0.86      6250

    accuracy                           0.86     12500
   macro avg       0.86      0.86      0.86     12500
weighted avg       0.86      0.86      0.86     12500

Classification Report for SVM
              precision    recall  f1-score   support

           0       0.85      0.84      0.84      6250
           1       0.84      0.85      0.84      6250

    accuracy                           0.84     12500
   macro avg       0.84      0.84      0.84     12500
weighted avg       0.84      0.84      0.84     12500

Classification Report for RF
              precision    recall  f1-score   support

           0       0.81      0.84      0.83      6250
           1       0.84      0.80      0.82      6250

    accuracy                           0.82     12500
   macro avg       0.82      0.82      0

In [ ]:
# So these were various results that had been found out by applying various models and functions on the text Data